Analyse socio-économique et délinquance, 2012-2016

In [2]:
import pandas as pd
#importation du dataset
data = pd.read_csv("full_dataset_normalized_2.csv", encoding='utf-8', sep=',')

C:\Users\thiba\AppData\Local\Temp\ipykernel_33084\2112852922.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("full_dataset_normalized_2.csv", encoding='utf-8', sep=',')


In [3]:
# On se concentre sur l'année 2012, et on exclue les valeurs France_Métro

data_2012 = data[data['annee']==2012]

data_2012 = data_2012[~(data_2012['code_dep'] == 'France_Métro')]

# Problème avec l'espace marquant le millier pour la colonne effectifs de police, mal converti par pandas d'où la nécessité de cet ajustement :
data_2012['effectif_total_police_national'] = data_2012['effectif_total_police_national'].replace('\xa0', '').astype(int)

#data_2012['effectif_total_police_national'].iloc[60]

# code_dep et Date sont en Object, on les convertit pour bien avoir le même type pour chaque colonne : str pour code_dep, date type pour Date

data_2012['code_dep'] = data_2012['code_dep'].astype(str)
data_2012['Date'] = pd.to_datetime(data_2012['Date'])

In [4]:
data_2012.columns

Index(['Date', 'code_dep', 'Homicides', 'Vols',
       'Cambriolages à domicile ou violaiton de domicile',
       'Crimes ou délits sexuelles', 'Infractions liées aux stupéfiants',
       'Dégradations ou destructions de biens volontaires', 'Violences',
       'Falsifications et contrefaçons', 'Délits Économiques et Financiers',
       'Irégularités de Main-d'oeuvre', 'annee', 'Population', 'Chomage',
       'Taux de pauvreté-moins de 30', 'Taux de pauvreté-30 à 39 ans',
       'Taux de pauvreté-40 à 49 ans', 'Taux de pauvreté-50 à 59 ans',
       'Taux de pauvreté-60 à 74 ans', 'Taux de pauvreté-75 ans ou plus',
       'Median_salaire', 'Taux de pauvreté-Ensemble', 'Bac_g_réussite_%',
       'Bac_t_réussite_%', 'Bac_p_réussite_%', 'Bac_g_%', 'Bac_t_%', 'Bac_p_%',
       'non_peu_diplômés_non_scolarisée_<15ans', 'Superficie\r\n(km2)',
       'effectif_total_police_national', 'Homicides_per_population',
       'Vols_per_population',
       'Cambriolages à domicile ou violaiton de domici

In [5]:
df1 = data_2012[['Date', 'code_dep', 'Vols_per_population', 'effectif_total_police_national_per_population']].copy()

In [6]:
#df1.head()

In [7]:
# Grouper par 'code_dep', puis calculer la moyenne annuelle pour les deux colonnes "vols_per_population" et effectifs policiers par population
df2 = df1.groupby('code_dep').mean().copy()

# Réinitialiser l'index si vous voulez que 'code_dep' redevienne une colonne
df2 = df2.reset_index()

df2.drop('Date', axis=1, inplace=True)

In [8]:
#test
#df2.loc[df2['code_dep'] == '17'] # nota : bien penser à mettre les guillemets car de type str

In [9]:
correlation = df2['Vols_per_population'].corr(df2['effectif_total_police_national_per_population'])

In [10]:
correlation

0.7373749231576757

In [11]:
# Corrélation entre effectifs de police et différents taux de délits/crimes ainsi que chômage, salaire médian, plusieurs catégories de pauvreté

categories = [ 'Homicides_per_population',
       'Vols_per_population',
       'Cambriolages à domicile ou violaiton de domicile_per_population',
       'Crimes ou délits sexuelles_per_population',
       'Infractions liées aux stupéfiants_per_population',
       'Dégradations ou destructions de biens volontaires_per_population',
       'Violences_per_population',
       'Falsifications et contrefaçons_per_population',
       'Délits Économiques et Financiers_per_population',
       "Irégularités de Main-d'oeuvre_per_population",
       'Chomage_per_population',
        'Taux de pauvreté-Ensemble',
        'Median_salaire',
        'Taux de pauvreté-moins de 30',
        'Taux de pauvreté-30 à 39 ans'
]

# Préparation du DataFrame pour le calcul de corrélation
df_correlation = pd.DataFrame(index=categories, columns=['Corrélation avec effectif_total_police_national_per_population'])

for categorie in categories:
    # Calculer la corrélation pour chaque catégorie
    df = data_2012[['Date', 'code_dep', categorie, 'effectif_total_police_national_per_population']].copy()
    df = df.groupby('code_dep').mean().copy()
    # Réinitialiser l'index si vous voulez que 'code_dep' redevienne une colonne
    df = df.reset_index() 
    df.drop('Date', axis=1, inplace=True)
    
    correlation = df[categorie].corr(df['effectif_total_police_national_per_population'])
    df_correlation.loc[categorie, 'Corrélation avec effectif_total_police_national_per_population'] = correlation

# Afficher les résultats
print(df_correlation)

                                                   Corrélation avec effectif_total_police_national_per_population
Homicides_per_population                                                                     0.343389            
Vols_per_population                                                                          0.737375            
Cambriolages à domicile ou violaiton de domicil...                                           0.351741            
Crimes ou délits sexuelles_per_population                                                    0.598007            
Infractions liées aux stupéfiants_per_population                                             0.588526            
Dégradations ou destructions de biens volontair...                                           0.534849            
Violences_per_population                                                                     0.507529            
Falsifications et contrefaçons_per_population                                           

In [17]:
# Corrélation entre plusieurs catégories clés de délits/crimes et différentes variables socio-économiques

categories_infractions = ['Homicides_per_population',
       'Vols_per_population',
       'Cambriolages à domicile ou violaiton de domicile_per_population',
       'Crimes ou délits sexuelles_per_population',
       'Infractions liées aux stupéfiants_per_population',
       'Dégradations ou destructions de biens volontaires_per_population',
       'Violences_per_population',
       'Falsifications et contrefaçons_per_population',
       'Délits Économiques et Financiers_per_population',
       "Irégularités de Main-d'oeuvre_per_population"]


categories_socio_ecos = ['Chomage_per_population',
        'Taux de pauvreté-Ensemble',
        'Median_salaire',
        'Taux de pauvreté-moins de 30',
        'Taux de pauvreté-30 à 39 ans',
        'Taux de pauvreté-40 à 49 ans', 
        'Taux de pauvreté-50 à 59 ans',
        'Taux de pauvreté-60 à 74 ans',
        'Taux de pauvreté-75 ans ou plus',
         'density']

for cat_infra in categories_infractions :

    # Préparation du DataFrame pour le calcul de corrélation
    df_correlation2 = pd.DataFrame(index=categories_socio_ecos, columns=['Corrélation avec '+cat_infra])
    
    for categorie in categories_socio_ecos:
        # Calculer la corrélation pour chaque catégorie
        df = data_2012[['Date', 'code_dep', categorie, cat_infra]].copy()
        df = df.groupby('code_dep').mean().copy()
        # Réinitialiser l'index si vous voulez que 'code_dep' redevienne une colonne
        df = df.reset_index() 
        df.drop('Date', axis=1, inplace=True)
        
        correlation = df[categorie].corr(df[cat_infra])
        df_correlation2.loc[categorie, 'Corrélation avec '+cat_infra] = correlation
    
    # Afficher les résultats
    print(df_correlation2)


                                Corrélation avec Homicides_per_population
Chomage_per_population                                           0.334933
Taux de pauvreté-Ensemble                                        0.308455
Median_salaire                                                   0.153667
Taux de pauvreté-moins de 30                                     0.168971
Taux de pauvreté-30 à 39 ans                                      0.24443
Taux de pauvreté-40 à 49 ans                                     0.257833
Taux de pauvreté-50 à 59 ans                                     0.329106
Taux de pauvreté-60 à 74 ans                                     0.434931
Taux de pauvreté-75 ans ou plus                                  0.064825
density                                                          0.110168
                                Corrélation avec Vols_per_population
Chomage_per_population                                      0.309495
Taux de pauvreté-Ensemble                       

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Faire heatmap corrélation entre les deux listes catégories + effectifs de police

#Calcul du taux de corrélation entre ces deux taux 
.corr(method="pearson")

# Visualize the correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(, annot=True, cmap='coolwarm')
plt.title('Matrice de corrélation entre dégradations et effectifs de police ')
plt.show()